# Orbital Elements to Cartesian Coordinates

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np
import matplotlib.pyplot as plt
import time

# Aliases
keras = tf.keras

In [3]:
# Local imports
import kepler_sieve
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
# from tf_utils import Identity
# from r2b import VectorError
from orbital_element import make_data_orb_elt, make_dataset_elt_to_cfg, make_dataset_cfg_to_elt
from orbital_element import OrbitalElementToConfig, make_model_elt_to_cfg
from orbital_element import ConfigToOrbitalElement, make_model_cfg_to_elt
from orbital_element import MeanToEccentricAnomaly, MeanToTrueAnomaly

In [4]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [5]:
# Plot style 
plot_style()

In [6]:
# Lightweight serialization
fname = '../data/r2b/orbital_element.pickle'
vartbl = load_vartbl(fname)

### Orbital Element Definitions

The module `orbital_element.py` contains calculations related to traditional orbital elements.<br>
All conventions and variable names follow those used in the `rebound` N-body integrator package.<br>

![title](../figs/web/orbital-elements.png)

In this diagram, the orbital plane (yellow) intersects a reference plane (gray).<br>
For Earth-orbiting satellites, the reference plane is usually the Earth's equatorial plane, and for satellites in solar orbits it is the ecliptic plane.<br>
The intersection is called the line of nodes, as it connects the center of mass with the ascending and descending nodes.<br>
The reference plane, together with the vernal point (♈︎), establishes a reference frame.

**Discussion of Orbital Elements (Wikipedia)**

Two elements define the shape and size of the ellipse:

* **Eccentricity (e)**—shape of the ellipse, describing how much it is elongated compared to a circle (not marked in diagram).
* **Semimajor axis (a)**—the sum of the periapsis and apoapsis distances divided by two. For circular orbits, the semimajor axis is the distance between the centers of the bodies, not the distance of the bodies from the center of mass.

Two elements define the orientation of the orbital plane in which the ellipse is embedded:

* **Inclination (i)**—vertical tilt of the ellipse with respect to the reference plane, measured at the ascending node (where the orbit passes upward through the reference plane, the green angle i in the diagram). Tilt angle is measured perpendicular to line of intersection between orbital plane and reference plane. Any three points on an ellipse will define the ellipse orbital plane. The plane and the ellipse are both two-dimensional objects defined in three-dimensional space.
* **Longitude of the ascending node (Ω)**—horizontally orients the ascending node of the ellipse (where the orbit passes upward through the reference plane, symbolized by ☊) with respect to the reference frame's vernal point (symbolized by ♈︎). This is measured in the reference plane, and is shown as the green angle Ω in the diagram.

The remaining two elements are as follows:

* **Argument of periapsis (ω)** defines the orientation of the ellipse in the orbital plane, as an angle measured from the ascending node to the periapsis (the closest point the satellite object comes to the primary object around which it orbits, the blue angle ω in the diagram).
* **True anomaly (ν, θ, or f) at epoch (M0)** defines the position of the orbiting body along the ellipse at a specific time (the "epoch").

The mean anomaly is a mathematically convenient "angle" which varies linearly with time, but which does not correspond to a real geometric angle. It can be converted into the true anomaly ν, which does represent the real geometric angle in the plane of the ellipse, between periapsis (closest approach to the central body) and the position of the orbiting object at any given time. Thus, the true anomaly is shown as the red angle ν in the diagram, and the mean anomaly is not shown.

The angles of inclination, longitude of the ascending node, and argument of periapsis can also be described as the Euler angles defining the orientation of the orbit relative to the reference coordinate system.

Note that non-elliptic trajectories also exist, but are not closed, and are thus not orbits. If the eccentricity is greater than one, the trajectory is a hyperbola. If the eccentricity is equal to one and the angular momentum is zero, the trajectory is radial. If the eccentricity is one and there is angular momentum, the trajectory is a parabola.

**Credit: Wikipedia** - https://en.wikipedia.org/wiki/Orbital_elements

**Variable Names of Six Keplerian Orbital Elements**
* `a` - semi-major axis; size of the ellipse; mean of periapsis and apoapsis distances
* `e` - eccentricity; shape of the ellipse.  e=0 is a circle, e=1 is a line.  $e^2 = 1 - b^2 / a^2$
* `inc` - inclination; angle between orbital and ecliptic planes
* `Omega` - longitude of the asending node; undefined when inc=0
* `omega` - argument of pericenter; true anomaly where body is closest to the primary.
* `f` - true anomaly; angle of the orbiting body in its orbital plane

**Two Additional Elements**
* `M` - mean anomaly; area swept out by the orbiting body, normalized so a full orbit is $2 \pi$.  Because of the rule of equal area in equal time, a.k.a. conservation of angular momentum, the mean anomly is linear in time
* `N` - mean motion; rate at which mean anomly changes, i.e. $2 \pi / T$ where $T$ is the orbital period

### Generating Reference Data Sets for Testing Orbital Element Calculations

The function `make_data_orb_elt` generates a data set used to test the conversion functions between Cartesian coordinates and orbital elements.  It initializes an orbit with parameters (`a`, `e`, `inc`, `Omega`, `omega`, `f`) in the Rebound package.<br>
* `a` is sampled uniformly in the range [`a_min`, `a_max`]
* `e` is sampled uniformly in the range [0, `e_max`]
* `inc` is sampled uniformly in the range [0, `inc_max`]
* `Omega`, `omega` and `f` are sampled uniformly in the range [$-\pi, +\pi$]
After a particle is initialized with these parameters, the Cartesian position and velocity are accessed and saved.  One entry in Cartesian configuration space has six coordinates: the x, y, and z coordinates of position and velocity.

The function `make_dataset_elt_to_cfg` wraps this data into a tensorflow dataset object whose inputs are orbital elements and whose outputs are in Cartesian configuration space.

The function `make_dataset_cfg_to_elt` wraps this data into a tensorflow dataset object whose inputs are Cartesian configurations and whose outputs are orbital elements.

In [7]:
# Create small data set for orbital elements; dictionaries of numpy arrays
n = 128
a_min = 1.0
a_max = 2.0
e_max = 1.0
inc_max = np.pi/4.0
seed=42
elts, cart = make_data_orb_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, inc_max=inc_max, seed=seed)

In [8]:
# Create a tensorflow Dataset instance in both directions
batch_size = 64
ds_e2c = make_dataset_elt_to_cfg(n=n, a_min=a_min, a_max=a_max, e_max=e_max, 
                                 inc_max=inc_max, seed=seed, batch_size=batch_size)
ds_c2e = make_dataset_cfg_to_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, 
                                 inc_max=inc_max, seed=seed, batch_size=batch_size)

In [9]:
# Example batch
elts, cart = list(ds_e2c.take(1))[0]

# Unpack orbital elements
a = elts['a']
e = elts['e']
inc = elts['inc']
Omega = elts['Omega']
omega = elts['omega']
f = elts['f']
mu = elts['mu']

# Unpack cartesian coordinates
q = cart['q']
v = cart['v']
acc = cart['acc']

# Review shapes
print(f'Example batch sizes:')
print(f'a    = {a.shape}')
print(f'e    = {e.shape}')
print(f'inc  = {inc.shape}')
print(f'Omega= {Omega.shape}')
print(f'omega= {omega.shape}')
print(f'f    = {f.shape}')
print(f'mu   = {f.shape}')
print(f'q    = {q.shape}')
print(f'v    = {v.shape}')
print(f'acc  = {acc.shape}')

Example batch sizes:
a    = (64,)
e    = (64,)
inc  = (64,)
Omega= (64,)
omega= (64,)
f    = (64,)
mu   = (64,)
q    = (64, 3)
v    = (64, 3)
acc  = (64, 3)


### Keras Layers & Models Converting Between Orbital Elements and Cartesian Configuration

The layer `ArcCos2` is a customized version of ArcCos matching one defined in Rebound.  It takes inputs `x`, `r` and `y` and returns an angle in the correct quadrant based on the sign of `y`.<br>
The layer `OrbitalElementToConfig` takes as inputs six orbital elements (`a`, `e`, `inc`, `Omega`, `omega`, `f`) and returns six Cartesian coordinates (`qx`, `qy`, `qz`, `vx`, `vy`, `vz`).  It is relatively straightforward and consists mainly of sums and products of trigonmetric functions.  It is ported almost verbatim from the Rebound library into Tensorflow.<br>
The layer `ConfigToOrbitalElement` transforms a Cartesian configuration into orbital elements.  This is not as straightforward because some configurations can be described in multiple ways.  For example, in a circular orbit, the longitude of the ascending node `Omega` is not uniquely defined.  A subset of the code in Rebound was ported.  Special edge cases were ignored, but the main case was ported over.  This appears to work fine for orbits of the types being studied here.<br>
A Keras layer is similar to a model, but a model has additional functionality.  Here we want to test whether the custom layers are matching the calculations done in Rebound, so we want to use the evaluate method on a model.<br>
The function `make_model_elt_to_cfg` wraps the `OrbitalElementToConfig` layer into a Keras model.<br>
The function `make_model_cfg_to_elt` wraps the `ConfigToOrbitalElement` layer into a Keras model.

In [10]:
# Create a model mapping orbital elements to cartesian coordinates
model_e2c = make_model_elt_to_cfg(include_accel=True)

### Test Conversion From Orbital Elements to Cartesian Configuration

In [11]:
# Inputs to compile this model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss'),
        'acc': VectorError(name='acc_loss'),}

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0,
                'acc': 1.0}

NameError: name 'VectorError' is not defined

In [ ]:
# Compile the e2c model
model_e2c.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [ ]:
# Summary of the model mapping orbital elements to position
model_e2c.summary()

In [ ]:
# Verify that model matches rebound
model_e2c.evaluate(ds_e2c)

In [ ]:
q_out, v_out, acc_out = model_e2c([a, e, inc, Omega, omega, f, mu])
traj_shape = (64, 3,)
q_out = tf.reshape(q_out, traj_shape)
v_out = tf.reshape(v_out, traj_shape)
acc_out = tf.reshape(acc_out, traj_shape)

**The model converts from orbital elements to Cartesian coordinates with very high accuracy, on the order of 1E-14 in the MSE.**<br>
Since the error here is squared, this is line with the full single precision of 7-8 decimal places.

We can also run the layer directly, as below:

In [ ]:
# Run the layer on the batch of orbital elements
inputs_e2c = (a, e, inc, Omega, omega, f, mu)
cart_rec = OrbitalElementToConfig()(inputs_e2c)

### Test Conversion From Cartesian Configuration to Orbital Elements

In [ ]:
# Create a model mapping cartesian coordinates to orbital elements
model_c2e = make_model_cfg_to_elt()

In [ ]:
# Inputs to compile the c2e model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'a': keras.losses.MeanSquaredError(),
        'e': keras.losses.MeanSquaredError(),
        'inc': keras.losses.MeanSquaredError(),
        'Omega': keras.losses.MeanSquaredError(),
        'omega': keras.losses.MeanSquaredError(),
        'f': keras.losses.MeanSquaredError(),
       }

metrics = None

loss_weights = {'a': 1.0,
                'e': 1.0,
                'inc': 1.0,
                'Omega': 1.0,
                'omega': 1.0,
                'f': 1.0}

In [ ]:
model_c2e.summary()

In [ ]:
# Compile the c2e model
model_c2e.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [ ]:
model_c2e.evaluate(ds_c2e)

**The model converts from Cartesian coordinates to orbital elements with very high accuracy, on the order of 1E-13 to 1E-14 in the MSE.**<br>
Since the error here is squared, this is approaching full single precision of 7-8 decimal places.

We can also run the `ConfigToOrbitalElement` layer directly, as below:

In [ ]:
# Run the layer on the batch of orbital elements
qx = q[:,0]
qy = q[:,1]
qz = q[:,2]
vx = v[:,0]
vy = v[:,1]
vz = v[:,2]
inputs_c2e = (qx, qy, qz, vx, vy, vz, mu)
elt_rec = ConfigToOrbitalElement()(inputs_c2e)

In [ ]:
# Review shapes
print(f'Example batch sizes:')
print(f'qx   = {qx.shape}')
print(f'vx   = {vx.shape}')
print(f'mu   = {mu.shape}')

### Conversion from Mean Anomaly to True Anomaly

One major advantage of Keplerian orbital elements is that they allow for easy calculation orbits.  Of the six orbital elements, 5 of them remain constant during the evolution of the restricted two body problem.  Only the true anomaly `f` (or equivalently the mean anomaly `M`) change.  The parameters `a`, `e`, `inc`, `Omega` and `omega` all remain constant.<br>
The mean anomaly `M` changes linearly with time, making it simple to evolve. In order to recover the predicted Cartesian coordinates though, we need a way to convert from the mean anomaly `M` back to the true anomaly `f`.  This can be done in two stages.<br>
The mean anomly `M` is related to the eccentric anomaly `E` and the eccentricity `e` by **Kepler's Equation**:
$$ M = E - e \sin E$$
See https://en.wikipedia.org/wiki/Kepler%27s_equation<br>
The calculation from `E` to `M` can be done in closed form according to the above formula.  The inverse function cannot be computed symbolically, but a numerical approach such as Newton's method will converge rapidly.  This is the method used in the layer `MeanToEccentricAnomaly`.  The code is ported from Rebound.  There is some subtlety involved in using Tensorflow operations (tf.cond must be used in place of an if statement) but it's not too complicated.<br>
The second step is to convert from the eccentric anomaly `E` to the true anomaly `f`.  This is easy and accomplished by a one liner:
$$ \tan \frac{f}{2} = \sqrt{\frac{1 + e}{1-e}} \cdot \tan \frac{E}{2}$$
This formula is used in the layer `MeanToTrueAnomaly`

In [ ]:
a, e, inc, Omega, omega, f, M, N = elt_rec

In [ ]:
# Test the mean anomaly convervsion functions
E = MeanToEccentricAnomaly()((M, e))
f_rec = MeanToTrueAnomaly()((M, e))

# Compute the RMS error
two_pi = 2.0 * np.pi
f_err = (f_rec % two_pi) - (f % two_pi)
rms_err = np.sqrt(np.mean(f_err * f_err))
print(f'RMS error of true anomaly f from mean anomaly M:')
print(f'{rms_err:5.2E}')

**The calculation recovers the starting true anomaly f to an RMS on the order of 1E-6**<br>
This could be slightly improved by using more iterations, but already accurate enough for the intended use case.